In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pickle
import functools
import gc
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
import lightgbm as lgb
import sklearn


In [2]:
############################preprocessing member file##########################################################
def preprocess_member(df,msno_file):
    """
    Objective: Preprocess the membes detail file
    
    Input: Membership data frame, train or test data frame
    
    OutPut: preprocessed members deetail
    """
    data = pd.merge(msno_file.reset_index(drop=True),df,on='msno',how='left')
    (city_vector,gender_vector,registered_via_vector) = pickle.load(open("member.pickle","rb"))
    #1. city
    data_city = city_vector.transform(list(map(str,data['city'].values.astype(int))))
    #2. bd
    #fill the nan value with -1 in age data
    data['bd'] = data['bd'].fillna(-1)
    # remove all the ages greater than 85 and lesser than 10 in train data
    data['bd'] = list(map(lambda x: -1 if x<10 or x>85 else x,data.bd))
    #3. gender
    data_gender = gender_vector.transform(list(map(str,data['gender'].values)))
    #4. registered_via
    data_registered_via = registered_via_vector.transform(list(map(str,data['registered_via'].values.astype(int))))
    #5. Registration init time 
    data['registration_init_time'] = data['registration_init_time'].fillna(-1)
    #stacking
    data = np.hstack((data[['bd','registration_init_time']].to_numpy(),data_city.todense(),data_gender.todense(),data_registered_via.todense()))   
    return data

In [3]:
def sum_feature(files,msno_file,name,last_day,month,days):
    features = ['num_25', 'num_50', 'num_75', 'num_985', 'num_100',
          'num_unq','total_secs']
    if len(files)>5:
        f = [files[:5],files[5:]]
    else:
        f = [files]
    merge = []
    for c in range(0,len(f)):
        #get features from log files from 2016
        merge_files = []
        for i in tqdm(features[:-1]):
            #load all the files we made from last iteration. But only loading pariticular columns to avoid memor constraints
            df = pd.concat([pd.read_csv(f,usecols=['msno',i,'date']) for f in f[c]])
            #to avoid leakage problem we only slecing data untill 2016
            df = pd.merge(pd.read_csv(msno_file).drop(['Unnamed: 0','is_churn'],axis=1),df,on='msno',how='left')
            gc.collect()
            df=df[df['date'].le(last_day)][['msno',i]]
            #creating sum,mean,std of a msno feature
            df=df.groupby('msno').agg({i:['sum','mean']})
            #rename the column name as ater group and aggregation names might be diffrent
            df.columns = df.columns = ["_".join(x) for x in df.columns.ravel()]
            #export our feature as csv
            df.to_csv(str(i)+'_'+str(c)+'_'+name+".csv")
            #clear the ram for next iteration
            merge_files.append(str(i)+'_'+str(c)+'_'+name+".csv")
            del df
            gc.collect()
        ##################################################################################################################
        #load all the files we made from last iteration. But only loading pariticular columns to avoid memory  constraints
        df = pd.concat([pd.read_csv(f,usecols=['msno','total_secs','date']) for f in f[c]])
        #to avoid leakage problem we only slecing data untill 2016
        df = pd.merge(pd.read_csv(msno_file).drop(['Unnamed: 0'],axis=1),df,on='msno',how='left')
        gc.collect()
        df=df[df['date'].le(last_day)][['msno','total_secs']]
        #creating sum,mean,std of a msno feature
        df=df.groupby('msno').agg({'total_secs':['sum','mean','count']})
        #rename the column name as ater group and aggregation names might be diffrent
        df.columns = df.columns = ["_".join(x) for x in df.columns.ravel()]
        #export our feature as csv
        df.to_csv('total_secs_'+str(c)+'_'+name+".csv")
        merge_files.append('total_secs_'+str(c)+'_'+name+".csv")
        #clear the ram for next iteration
        del df
        gc.collect()
        merge.append(merge_files)
    final_merge = []
    for i in range(len(features)):
        df = pd.concat([pd.read_csv(f[i]).rename(columns = {'msno_':'msno'}) for f in merge])
        val =dict()
        for j in list(df.columns):
            if 'sum' in j or 'count' in j:
                val[j]=['sum']
            elif 'mean' in j:
                val[j]=['mean']
        df = df.groupby('msno').agg(val)
        df.columns = df.columns = [x[0] for x in df.columns.ravel()]
        df = df.rename(columns = {'msno_':'msno'})
        df.to_csv(features[i]+'_{}.csv'.format(name))
        final_merge.append(features[i]+'_{}.csv'.format(name))
    data_frames = [pd.read_csv(msno_file).drop(['Unnamed: 0','is_churn'],axis=1)]
    data_frames += [pd.read_csv(f) for f in final_merge]
    df_merged = functools.reduce(lambda  left,right: pd.merge(left,right,on=['msno'],how='left'), data_frames)
    df_merged.to_csv("all_log_{}.csv".format(name))
    del df_merged
    gc.collect()

In [4]:
def last_month_features(files,msno_file,name,date,f_name):
    #sum_feature(files,msno_file,name,last_day,date,days)
    #last date features from log file
    features = ['num_25', 'num_50', 'num_75', 'num_985', 'num_100',
          'num_unq','total_secs']
    merge_files = []
    for i in tqdm(features[:-1]):
        #load all the files we made from last iteration. But only loading pariticular columns to avoid memor constraints
        temp = []
        for f in files:
            df = pd.read_csv(f,usecols=['msno',i,'date'])
            df = df[df['date'].between(date[0],date[1])][['msno',i]]
            temp.append(df)
            gc.collect()
        df = pd.concat(temp)
        df = pd.merge(pd.read_csv(msno_file).drop(['Unnamed: 0','is_churn'],axis=1),df,on='msno',how='left')
        #creating sum,mean,std of a msno feature
        df=df.groupby('msno').agg({i:['sum','mean','std']}).reset_index()
        #rename the column name as ater group and aggregation names might be diffrent
        df.columns = df.columns = ["_".join(x) for x in df.columns.ravel()]
        df.columns = df.columns = [f_name+"_"+x for x in df.columns]
        df = df.rename(columns = {f_name+"_msno_":'msno'})
        #export our feature as csv
        df.to_csv(f_name+'_'+str(i)+'_'+name+".csv")
        merge_files.append(f_name+'_'+str(i)+'_'+name+".csv")
        #clear the ram for next iteration
        del df
        gc.collect()    

    #load all the files we made from last iteration. But only loading pariticular columns to avoid memor constraints
    temp = []
    for f in files:
        df = pd.read_csv(f,usecols=['msno','total_secs','date'])
        df = df[df['date'].between(date[0],date[1])][['msno','total_secs']]
        temp.append(df)
        gc.collect()
    df = pd.concat(temp)
    df = pd.merge(pd.read_csv(msno_file).drop(['Unnamed: 0','is_churn'],axis=1),df,on='msno',how='left')
    #creating sum,mean,std of a msno feature
    df=df.groupby('msno').agg({'total_secs':['sum','mean','std','count']}).reset_index()
    #rename the column name as ater group and aggregation names might be diffrent
    df.columns = df.columns = ["_".join(x) for x in df.columns.ravel()]
    df.columns = df.columns = [f_name+"_"+x for x in df.columns]
    df = df.rename(columns = {f_name+"_msno_":'msno'})
    #export our feature as csv
    df.to_csv(f_name+'_total_secs_'+name+".csv")
    merge_files.append(f_name+'_total_secs_'+name+".csv")
    #clear the ram for next iteration
    del df
    gc.collect()   
    data_frames = [pd.read_csv(msno_file).drop(['Unnamed: 0','is_churn'],axis=1)]
    data_frames += [pd.read_csv(f).drop(['Unnamed: 0'],axis=1) for f in merge_files]    
    df_merged = functools.reduce(lambda  left,right: pd.merge(left,right,on=['msno'],how='left'), data_frames)
    df_merged.to_csv("{}_log_{}.csv".format(f_name,name))
    del df_merged
    gc.collect()

In [5]:
def Final_day_features(files,msno_file,name,date,f_name):
    #sum_feature(files,msno_file,name,last_day,date,days)
    #last date features from log file
    features = ['num_25', 'num_50', 'num_75', 'num_985', 'num_100',
          'num_unq','total_secs']
    merge_files = []
    for i in tqdm(features):
        #load all the files we made from last iteration. But only loading pariticular columns to avoid memor constraints
        temp = []
        for f in files:
            df = pd.read_csv(f,usecols=['msno',i,'date'])
            df = df[df['date']==date][['msno',i]]
            temp.append(df)
            gc.collect()
        df = pd.concat(temp)
        df = pd.merge(pd.read_csv(msno_file).drop(['Unnamed: 0','is_churn'],axis=1),df,on='msno',how='left')
        #rename the column name as ater group and aggregation names might be diffrent
        df.columns = df.columns = [f_name+"_"+x for x in df.columns]
        df = df.rename(columns = {f_name+"_msno":'msno'})
        #export our feature as csv
        df.to_csv(f_name+'_'+str(i)+'_'+name+".csv")
        merge_files.append(f_name+'_'+str(i)+'_'+name+".csv")
        #clear the ram for next iteration
        del df
        gc.collect()    
    data_frames = [pd.read_csv(msno_file).drop(['Unnamed: 0','is_churn'],axis=1)]
    data_frames += [pd.read_csv(f).drop(['Unnamed: 0'],axis=1) for f in merge_files]    
    df_merged = functools.reduce(lambda  left,right: pd.merge(left,right,on=['msno'],how='left'), data_frames)
    df_merged = df_merged.drop_duplicates(subset=['msno'], keep="first", inplace=False)
    df_merged.to_csv("{}_log_{}.csv".format(f_name,name))
    del df_merged
    gc.collect()

In [6]:
def preprocess_log(files,msno_file,name,last_day,month,days):
    """
    Objective: Preprocess the user_log file and create a sum,avg,std features for given columns
    
    Input: file:transaction_filename,msno_file=train or test file name,
    name:name of the file we preprocess,last_day:transaction last day,month:transaction last month,lastday:last 15 days
    
    OutPut: preprocessed members deetail
    """
    sum_feature(files,msno_file,name,last_day,month,days)
    last_month_features(files,msno_file,name,month,'last_month')
    last_month_features(files,msno_file,name,days,'last_15days')
    Final_day_features(files,msno_file,name,last_day,'final_day')
#     merge_files = [msno_file,'Processed/all_log_{}.csv'.format(name),"Processed/last_month_log_{}.csv".format(name),"Processed/last_15days_log_{}.csv".format(name),"Processed/final_day_log_{}.csv".format(name)]
#     data_frames = [pd.read_csv(f).drop(['Unnamed: 0'],axis=1) for f in merge_files]    
#     df_merged = functools.reduce(lambda  left,right: pd.merge(left,right,on=['msno'],how='left'), data_frames)
#     df_merged.to_csv("Final/Final_log_{}.csv".format(name))
#     del df_merged
    gc.collect()

In [7]:
#Reference https://github.com/G-pravin-shankar/Kaggle-Top-4-percent-Solution-WSDM-KKBOX-Churn-Prediction/
def last_5_calc(msno_file,msno,last_5,last =-1):
    """
    Objective: Compute last five churn list
    Input : datafram with transaction_date and membership_expire date, size of train set, msno of trainset
    Output: return dictionary with msno key and last five churn details as value list
    """
    dic =dict(zip(pd.read_csv(msno_file)['msno'],np.arange(len(pd.read_csv(msno_file)['msno']))))
    Last_5_values = np.full((pd.read_csv(msno_file).shape[0], 5), last) #default value -1 meaning no transaction. Also index 5=last1 churn(becasue of asecending order)
    count=0
    index=-1
    for i in range(msno.shape[0]):
        ind=dic[msno[i]]
        if ind==index:
            count+=1
        else: #arrange values in right position and change 'count' to 0
            if count==3:
                Last_5_values[index][4] = Last_5_values[index][3]
                Last_5_values[index][3] = Last_5_values[index][2]
                Last_5_values[index][2] = Last_5_values[index][1]
                Last_5_values[index][1] = Last_5_values[index][0]
                Last_5_values[index][0]=-1
            elif count==2:
                Last_5_values[index][4] = Last_5_values[index][2]
                Last_5_values[index][3] = Last_5_values[index][1]
                Last_5_values[index][2] = Last_5_values[index][0]
                Last_5_values[index][1]= -1
                Last_5_values[index][0]= -1
            elif count==1:
                Last_5_values[index][4] = Last_5_values[index][1]
                Last_5_values[index][3] = Last_5_values[index][0]
                Last_5_values[index][1] = -1
                Last_5_values[index][0] = -1
            elif count==0:
                Last_5_values[index][4] = Last_5_values[index][0]
                Last_5_values[index][0] = -1
            count=0
        index=ind
        Last_5_values[index][count] = last_5[i]
    return Last_5_values

In [8]:
def Transaction_preprocess(files, msno_file, last_date ,name, date):
    """
    Objective : Preprocess the transaction file and save features as csv
    Input     : File names of transaction file, File name of msno file, name of set,last transactio date, last membership expire date
    Output    : save all the preprocessed features as csv
    """
    train = pd.read_csv(msno_file).drop(['Unnamed: 0'],axis=1)
    trans = pd.concat([pd.read_csv(f) for f in files])
    trans_before_last_date = trans[trans.transaction_date.le(last_date)]
    trans_before_last_date = trans_before_last_date[trans_before_last_date['membership_expire_date'].le(date[1])]
    trans_before_last_date = trans_before_last_date[trans_before_last_date.msno.isin(train['msno'])]
    trans_before_last_date = trans_before_last_date.sort_values(by=['msno', 'transaction_date'],ascending=[True,True])
    #1. total number of cancel feature
    print("1. total number of cancel feature")
    total_cancel = trans_before_last_date.groupby(['msno'], as_index=False)['is_cancel'].sum()
    total_cancel = pd.merge(train, total_cancel, on='msno',how='left')
    np.save('{}_total_num_cancel.npy'.format(name),list(total_cancel['is_cancel']))
    #2. Last Before transaction features
    print("2. Last Before transaction features")
    #https://stackoverflow.com/a/19464054
    index = trans_before_last_date[['msno','membership_expire_date','transaction_date','is_cancel']].shift(1) != trans_before_last_date[['msno','membership_expire_date','transaction_date','is_cancel']]
    trans_before_last_date = trans_before_last_date[index.apply(lambda x: (x[0] or x[1]) or (x[2] or x[3]),axis=1)]
    #https://stackoverflow.com/a/38965036
    las_before_transaction = trans_before_last_date.groupby(['msno']).tail(2).drop_duplicates(['msno'])
    las_before_transaction = pd.merge(train, las_before_transaction, on='msno',how='left')
    only_one_trans = trans_before_last_date.msno.value_counts(sort=False)
    index  = np.where(only_one_trans == 1)[0]
    one_trans = list(map(lambda x:only_one_trans.index[x],index))
    las_before_transaction[las_before_transaction['msno'].isin(one_trans)] = np.nan
    las_before_transaction=las_before_transaction.rename(columns={'payment_method_id': 'Last_before_payment_method_id', 'payment_plan_days': 'Last_before_payment_plan_days',\
                                                         'plan_list_price':'Last_before_plan_list_price','actual_amount_paid':'Last_before_actual_amount_paid',\
                                                         'is_auto_renew':'Last_before_is_auto_renew','transaction_date':'Last_before_transaction_date',\
                                                         'membership_expire_date':'Last_before_membership_expire_date','is_cancel':'Last_before_is_cancel'})
    las_before_transaction =  las_before_transaction.drop(['msno'], axis=1)
    las_before_transaction.to_csv('{}_last_before_transact_features_msno_order'.format(name))
    #3. Final transaction feature
    print("3. Final transaction feature")
    final_transaction = trans_before_last_date.groupby('msno').tail(1)
    final_transaction = pd.merge(train.msno, final_transaction, on='msno',how='left')
    final_transaction[final_transaction['membership_expire_date']<date[0]]
    final_transaction.to_csv('{}_final_transact.csv'.format(name))
    # Last 5 Churn and total churn and not churn rate
    print(" Last 5 Churn and total churn and not churn rate")
    trans_before_date_up = trans_before_last_date
    trans_before_date_up['Transaction_date_moved_up'] = list(trans_before_last_date['transaction_date'])[1:]+[date[0]]
    churn_rows = trans_before_date_up[['membership_expire_date','Transaction_date_moved_up']]
    churn = churn_rows.apply(lambda x: (((x[1]//10000*365)+(x[1]%10000//100*30.4167)+(x[1]%100))-((x[0]//10000*365)+(x[0]%10000//100*30.4167)+(x[0]%100)))<31, axis=1)
    last_not_churn = list(churn)
    np.save('churn_or_not_{}.npy'.format(name),last_not_churn)
    last_not_churn = np.load('churn_or_not_{}.npy'.format(name))
    trans_before_date_up['churn_or_not'] = last_not_churn
    ind=trans_before_date_up.groupby(['msno']).tail(1)
    #trans_before_date_up = trans_before_date_up[~trans_before_date_up.isin(ind)] its not working when we have duplicates in test
    #https://stackoverflow.com/questions/38681340/how-to-remove-common-rows-in-two-dataframes-in-pandas#:~:text=4%20Answers&text=You%20can%20use%20pandas.,the%20duplicated%20rows%20in%20them.

    df = trans_before_date_up.drop_duplicates().merge(ind.drop_duplicates(), on=ind.columns.to_list(), 
                   how='left', indicator=True)
    trans_before_date_up = df.loc[df._merge=='left_only',df.columns!='_merge']
    #4. Total_churn_and_not_churn_count:
    print("4. Total_churn_and_not_churn_count:")
    trans_before_date_up_ = trans_before_date_up.dropna()[['msno','churn_or_not']]
    msno = trans_before_date_up_['msno'].to_numpy()
    churn_or_not = trans_before_date_up_['churn_or_not'].to_numpy()
    dic =dict(zip(pd.read_csv(msno_file)['msno'],np.arange(len(pd.read_csv(msno_file)['msno']))))
    total_churn_or_not_count=np.full((pd.read_csv(msno_file).shape[0],2), 0)
    for i in tqdm(range(trans_before_date_up_.shape[0])):
        index = dic[msno[i]]
        if churn_or_not[i]==1: #not churn
            total_churn_or_not_count[index][1]+=1
        else: #churn
            total_churn_or_not_count[index][0]+=1
    np.save('total_churn_or_not_count_{}'.format(name),total_churn_or_not_count)
    #5. Last_5_not_churns:
    print("5. Last_5_not_churns:")
    trans_before_date_up = trans_before_date_up.groupby(['msno']).tail(5)
    trans_before_date_up = trans_before_date_up.dropna()
    trans_before_date_up = trans_before_date_up[['msno','churn_or_not']]
    msno = trans_before_date_up['msno'].to_numpy()
    churn_or_not = trans_before_date_up['churn_or_not'].to_numpy()
    Last_5_churn_or_not = last_5_calc(msno_file,msno,churn_or_not)
    np.save('Last_5_churn_or_not_{}'.format(name),Last_5_churn_or_not)
    # additinal feature genration
    print("additinal feature genration")
    transactions = pd.concat([pd.read_csv(f) for f in files])
    trans_before_last_date = transactions[transactions['transaction_date'].le(last_date)]
    trans_before_last_date = trans_before_last_date[trans_before_last_date['msno'].isin(train['msno'])]
    trans_before_last_date = trans_before_last_date.sort_values(by=['msno', 'transaction_date'],ascending=[True,True])
    trans_before_last_date = trans_before_last_date[trans_before_last_date['is_cancel']==0]
    #6.Discount feature 
    print("6. Discount feature ")
    trans_before_last_date['discount'] = trans_before_last_date['plan_list_price']-trans_before_last_date['actual_amount_paid']
    trans_before_last_date['discount'] = list(map(lambda x: -1 if x<0 else x,tqdm(trans_before_last_date['discount'])))
    #7pay per day
    print("7. pay per day")
    trans_before_last_date['per_day_price'] = trans_before_last_date['plan_list_price']/trans_before_last_date['payment_plan_days']
    #impute missing values with -1
    trans_before_last_date['per_day_price'] = trans_before_last_date['per_day_price'].fillna(-1)
    index = trans_before_last_date[['msno','membership_expire_date']].shift(1) !=trans_before_last_date[['msno','membership_expire_date']]
    trans_before_last_date = trans_before_last_date[index.apply(lambda x: (x[0] or x[1]),axis = 1)]
    #8. Sum_count_and_average_features:
    print("8. Sum_count_and_average_features:")
    sum_feature = trans_before_last_date.groupby(['msno'], as_index=False)['actual_amount_paid','payment_plan_days','discount','per_day_price'].sum()
    count_feature = trans_before_last_date.groupby(['msno'], as_index=False)['actual_amount_paid','payment_plan_days','discount','per_day_price'].count()
    count_feature[['actual_amount_paid_avg','payment_plan_days_avg','discount_avg','per_day_price_avg']] = sum_feature[['actual_amount_paid', 'payment_plan_days', 'discount','per_day_price']]/count_feature[['actual_amount_paid','payment_plan_days','discount','per_day_price']]
    count_feature = count_feature[['msno','actual_amount_paid','actual_amount_paid_avg','payment_plan_days_avg','discount_avg','per_day_price_avg']]
    count_feature.columns=['msno','count','actual_amount_paid_avg','payment_plan_days_avg','discount_avg','per_day_price_avg']
    count_feature['discount_avg']=list(map(lambda x: -1 if x<0 else x,tqdm(count_feature['discount_avg'])))
    count_feature['per_day_price_avg']=list(map(lambda x: -1 if x<0 else x,tqdm(count_feature['per_day_price_avg'])))
    sum_feature = sum_feature.merge(count_feature,on='msno')
    sum_feature = pd.DataFrame(train['msno'],columns=['msno']).merge(sum_feature,on='msno',how='left')
    sum_feature = sum_feature.fillna(-1)
    sum_feature = sum_feature.drop(['msno'],axis=1)
    np.save('transact_sum_count_avg_features_{}'.format(name),np.array(sum_feature))
    #last five features
    print("9. slast five features")
    trans_before_last_date_5 = trans_before_last_date.groupby(['msno']).tail(5)
    msno=trans_before_last_date_5['msno'].to_numpy()
    Last_5_discount = trans_before_last_date_5['discount'].to_numpy()
    Last_5_payment_plan_days=trans_before_last_date_5['payment_plan_days'].to_numpy()
    Last_5_actual_amount_paid=trans_before_last_date_5['actual_amount_paid'].to_numpy()
    Last_5_per_day_price=trans_before_last_date_5['per_day_price'].to_numpy()
    Last_5_discount = last_5_calc(msno_file,msno,Last_5_discount)
    np.save('Last_5_discount_{}'.format(name),Last_5_discount)
    Last_5_payment_plan_days = last_5_calc(msno_file,msno,Last_5_payment_plan_days)
    np.save('Last_5_payment_plan_days_{}'.format(name),Last_5_payment_plan_days)
    Last_5_actual_amount_paid = last_5_calc(msno_file,msno,Last_5_actual_amount_paid)
    np.save('Last_5_actual_amount_paid_{}'.format(name),Last_5_actual_amount_paid)
    Last_5_per_day_price = last_5_calc(msno_file,msno,Last_5_per_day_price,last =-1.0)
    np.save('Last_5_per_day_price_{}'.format(name),Last_5_per_day_price)
    #Final Transaction Additional Features
    print("10. Final Transaction Additional Features")
    final_trans = pd.read_csv("{}_final_transact.csv".format(name))
    #https://stackoverflow.com/questions/27506367/python-pandas-integer-yyyymmdd-to-datetime
    final_trans['transaction_date'] = pd.to_datetime(final_trans['transaction_date'].astype(str), format='%Y%m%d')
    final_trans['membership_expire_date']=pd.to_datetime(final_trans['membership_expire_date'].astype(str), format='%Y%m%d')
    #number of days from transaction date to membership expire date
    print("11. number of days from transaction date to membership expire date")
    Expiry_Trans_interval = final_trans['membership_expire_date']-final_trans['transaction_date']
    #https://stackoverflow.com/a/18215499
    Expiry_Trans_interval = Expiry_Trans_interval/np.timedelta64(1, 'D')
    #impute -1 for missing values
    Expiry_Trans_interval[Expiry_Trans_interval<0]=-1
    np.save('Expiry_Transaction_interval_{}'.format(name),Expiry_Trans_interval)
    #Days_since_final_transact:
    print("12. Days_since_final_transact:")
    Days_since_final_trans = pd.to_datetime(date[0],format='%Y%m%d') - final_trans['transaction_date']
    Days_since_final_trans = Days_since_final_trans/np.timedelta64(1, 'D')
    np.save('Days_since_final_transact_{}'.format(name),Days_since_final_trans)
    #Days left
    print("13. Days left")
    days_Left = final_trans['membership_expire_date']-pd.to_datetime(last_date,format='%Y%m%d')
    days_Left = days_Left/np.timedelta64(1, 'D')
    np.save('days_Left_{}'.format(name),days_Left)
    #Payment_plan_days_subract_interval:
    print("14. Payment_plan_days_subract_interval:")
    final_trans['Expiry_Transaction_interval']=Expiry_Trans_interval
    Expiry_Trans_interval-final_trans['payment_plan_days']
    Payment_plan_days_subract_interval = Expiry_Trans_interval - final_trans['payment_plan_days']
    np.save('Payment_plan_days_subract_interval_{}'.format(name),Payment_plan_days_subract_interval)
    # https://github.com/G-pravin-shankar/Kaggle-Top-4-percent-Solution-WSDM-KKBOX-Churn-Prediction/blob/master/Transcation_based_features_additional_train.ipynb
    Days_Left=np.load('days_Left_{}.npy'.format(name))
    Days_since_final_transact=np.load('Days_since_final_transact_{}.npy'.format(name))
    Expiry_Transaction_interval=np.load('Expiry_Transaction_interval_{}.npy'.format(name))
    Payment_plan_days_subract_interval=np.load('Payment_plan_days_subract_interval_{}.npy'.format(name))
    Final_Add=np.hstack((Days_Left.reshape(-1,1),Days_since_final_transact.reshape(-1,1),Expiry_Transaction_interval.reshape(-1,1),Payment_plan_days_subract_interval.reshape(-1,1)))
    np.save('Final_Add_{}.npy'.format(name),Final_Add)

In [9]:
def prepare_data(name,date):
    #Load all log preprocessed data
    final_day_log = pd.read_csv("final_day_log_{}.csv".format(name)).drop([i for i in pd.read_csv("final_day_log_{}.csv".format(name)).columns if 'Unnamed' in i]+['msno'],axis=1).to_numpy()
    last_15_days_log = pd.read_csv("last_15days_log_{}.csv".format(name)).drop([i for i in pd.read_csv("last_15days_log_{}.csv".format(name)).columns if 'Unnamed' in i]+['msno'],axis=1).to_numpy()
    last_month_log = pd.read_csv("last_month_log_{}.csv".format(name)).drop([i for i in pd.read_csv("last_month_log_{}.csv".format(name)).columns if 'Unnamed' in i]+['msno'],axis=1).to_numpy()
    all_month_log = pd.read_csv("all_log_{}.csv".format(name)).drop([i for i in pd.read_csv("all_log_{}.csv".format(name)).columns if 'Unnamed' in i]+['msno'],axis=1).to_numpy()
    last_month_15days_subtract_log = last_month_log-last_15_days_log
    #stack all the preprocessed fetures of log 
    user_log=np.hstack((final_day_log,last_month_15days_subtract_log,last_15_days_log,last_month_log,all_month_log))
    #save the file for later use
    np.save("log_{}".format(name),user_log)
    print("user log",user_log.shape)
    #members file details
    members=np.load('{}_member.npy'.format(name))
    print("members",members.shape)
    #load all the features generated using transaction file
    total_churn_notchurn_count = np.load('total_churn_or_not_count_{}.npy'.format(name))
    last_5_churn_notchurns     = np.load('Last_5_churn_or_not_{}.npy'.format(name))
    total_cancel_feature       = np.load('{}_total_num_cancel.npy'.format(name))
    final_trans                = pd.read_csv('{}_final_transact.csv'.format(name))
    last_before_trans          = pd.read_csv('{}_last_before_transact_features_msno_order'.format(name))
    trans_sum_count_avg        = np.load('transact_sum_count_avg_features_{}.npy'.format(name))
    Last_5_actual_amount_paid  = np.load('Last_5_actual_amount_paid_{}.npy'.format(name))
    Last_5_discount            = np.load('Last_5_discount_{}.npy'.format(name))
    Last_5_payment_plan_days   = np.load('Last_5_payment_plan_days_{}.npy'.format(name))
    Last_5_per_day_price       = np.load('Last_5_per_day_price_{}.npy'.format(name))
    Final_Add                  = np.load('Final_Add_{}.npy'.format(name))
    #remove label feature and unnamed columns i,e., index
    last_before_trans = last_before_trans.drop([i for i in last_before_trans.columns if 'is_churn' in i or 'Unnamed' in i],axis=1)
    #to avoid over fitting converting a date feature into a number of days from last expire date
    last_before_trans['Last_before_membership_expire_date'] = pd.to_datetime(last_before_trans['Last_before_membership_expire_date'].astype(str),format='%Y%m%d')
    last_before_trans['Last_before_membership_expire_date'] = pd.to_datetime(date[0],format='%Y%m%d')-last_before_trans['Last_before_membership_expire_date']
    last_before_trans['Last_before_membership_expire_date'] = last_before_trans['Last_before_membership_expire_date']/np.timedelta64(1, 'D')
    last_before_trans['Last_before_transaction_date'] = pd.to_datetime(last_before_trans['Last_before_transaction_date'].astype(str),format='%Y%m%d')
    last_before_trans['Last_before_transaction_date'] = pd.to_datetime(date[0],format='%Y%m%d')-last_before_trans['Last_before_transaction_date']
    last_before_trans['Last_before_transaction_date'] = last_before_trans['Last_before_transaction_date']/np.timedelta64(1, 'D')
    #count vectorizing a payment feature
    final_trans = final_trans.drop(['Unnamed: 0','transaction_date','membership_expire_date'],axis=1)
    #Last Before Count Vectorizer
    if name =='train':
        #https://stackoverflow.com/a/33264704
        payment_method_id = CountVectorizer( lowercase=False, binary=True, token_pattern = r"(?u)\b\w+\b")
        payment_method_id.fit(list(map(str, last_before_trans['Last_before_payment_method_id'].values.astype(int))))
        auto_renew = CountVectorizer( lowercase=False, binary=True, token_pattern = r"(?u)\b\w+\b")
        auto_renew.fit(list(map(str, last_before_trans['Last_before_is_auto_renew'].values.astype(int))))
        is_cancel = CountVectorizer( lowercase=False, binary=True, token_pattern = r"(?u)\b\w+\b")
        is_cancel.fit(list(map(str, last_before_trans['Last_before_is_cancel'].values.astype(int))))
        pickle.dump((payment_method_id,auto_renew,is_cancel),open("last_before_trans.pickle","wb"))
        last_before_payment_method = payment_method_id.transform(list(map(str,last_before_trans['Last_before_payment_method_id'].values.astype(int))))
        last_before_auto_renew = auto_renew.transform(list(map(str,last_before_trans['Last_before_is_auto_renew'].values.astype(int))))
        last_before_cancel =is_cancel.transform(list(map(str,last_before_trans['Last_before_is_cancel'].values.astype(int))))
    else:
        payment_method_id,auto_renew,is_cancel = pickle.load(open("last_before_trans.pickle","rb"))
        last_before_payment_method = payment_method_id.transform(list(map(str,last_before_trans['Last_before_payment_method_id'].values.astype(int))))
        last_before_auto_renew = auto_renew.transform(list(map(str,last_before_trans['Last_before_is_auto_renew'].values.astype(int))))
        last_before_cancel =is_cancel.transform(list(map(str,last_before_trans['Last_before_is_cancel'].values.astype(int))))
    last_before_trans = last_before_trans.drop(['Last_before_payment_method_id','Last_before_is_auto_renew','Last_before_is_cancel'],axis=1)
    last_before_trans = np.array(last_before_trans)
    last_before_trans = np.hstack((last_before_payment_method.todense(),last_before_auto_renew.todense(),last_before_trans,last_before_cancel.todense()))
    # Count vectorizing a final day payemnt id feature
    #Final Count Vectorizer
    if name =='train':
        #https://stackoverflow.com/a/33264704
        fpayment_method_id = CountVectorizer( lowercase=False, binary=True, token_pattern = r"(?u)\b\w+\b")
        val = list(final_trans['payment_method_id'])
        val[0] = np.nan
        fpayment_method_id.fit(list(map(str, np.array(val).astype(int))))
        fauto_renew = CountVectorizer( lowercase=False, binary=True, token_pattern = r"(?u)\b\w+\b")
        val = list(final_trans['is_auto_renew'])
        val[0] = np.nan
        fauto_renew.fit(list(map(str, np.array(val).astype(int))))
        fis_cancel = CountVectorizer( lowercase=False, binary=True, token_pattern = r"(?u)\b\w+\b")
        val = list(final_trans['is_cancel'])
        val[0] = np.nan
        fis_cancel.fit(list(map(str, np.array(val).astype(int))))
        pickle.dump((fpayment_method_id,fauto_renew,fis_cancel),open("final_trans.pickle","wb"))
        payment_method = fpayment_method_id.transform(list(map(str,final_trans['payment_method_id'].values.astype(int))))
        auto_renew = fauto_renew.transform(list(map(str,final_trans['is_auto_renew'].values.astype(int))))
        cancel = fis_cancel.transform(list(map(str,final_trans['is_cancel'].values.astype(int))))
    else:
        fpayment_method_id,fauto_renew,fis_cancel = pickle.load(open("final_trans.pickle","rb"))
        payment_method = fpayment_method_id.transform(list(map(str,final_trans['payment_method_id'].values.astype(int))))
        auto_renew = fauto_renew.transform(list(map(str,final_trans['is_auto_renew'].values.astype(int))))
        cancel = fis_cancel.transform(list(map(str,final_trans['is_cancel'].values.astype(int))))
    final_trans = final_trans.drop(['msno','payment_method_id','is_auto_renew','is_cancel'],axis=1)
    final_trans = np.array(final_trans)
    final_trans = np.hstack((payment_method.todense(),auto_renew.todense(),cancel.todense(),final_trans))
    #churn rate feature
    churn_rate=total_churn_notchurn_count[:,0]/(total_churn_notchurn_count[:,0]+total_churn_notchurn_count[:,1])
    trans = np.hstack((Final_Add,trans_sum_count_avg,Last_5_actual_amount_paid,Last_5_discount,Last_5_payment_plan_days,Last_5_per_day_price,
                          total_cancel_feature.reshape(-1,1),last_5_churn_notchurns,total_churn_notchurn_count,last_before_trans,final_trans,churn_rate.reshape(-1,1)))
    np.save("trans_{}".format(name),trans)
    print("trans",trans.shape)
    #combine all the features
    X = np.hstack((user_log,members,trans))
    #convert its type to float to input nan
    X = X.astype(np.float64)
    #replace NaN with -1.0
    X[np.isnan(X)]=-1.0
    np.save('X_{}'.format(name),X) 
    print("X_{}".format(name),X.shape)

In [10]:
# msno = pd.read_csv("user_label_201702.csv")
# msno[msno.msno=='+2vC1rM36Emx77UanRb3cUohWiIf7knfVIDO2+R78BE='].to_csv("query_msno.csv")
# t = pd.read_csv("transactions.csv")
# trans = t[t.msno=="+2vC1rM36Emx77UanRb3cUohWiIf7knfVIDO2+R78BE="]
# trans.to_csv("query_trans.csv",index=False)
# m = pd.read_csv("members_v3.csv")
# m = m[m.msno=="+2vC1rM36Emx77UanRb3cUohWiIf7knfVIDO2+R78BE="]
# m.to_csv("query_member.csv",index=False)
# f = ["./chunk/user_logs_chunk_{}.csv".format(i) for i in range(10)]+["./data/churn_comp_refresh/user_logs_v2.csv"]
# l_temp = []
# for i in tqdm(f):
#     temp = pd.read_csv(i)
#     l_temp.append(temp[temp.msno=='+2vC1rM36Emx77UanRb3cUohWiIf7knfVIDO2+R78BE='])
# l = pd.concat(l_temp)
# l
# l.to_csv("query_log.csv",index=False)

100%|██████████| 11/11 [10:52<00:00, 59.35s/it]


In [17]:
from math import log
#https://www.kaggle.com/c/bioresponse/discussion/1831
def log_loss(predicted, target):
    if len(predicted) != len(target):
        print('lengths not equal!')
        return
    target = [float(x) for x in target]   # make sure all float values
    predicted = [min([max([x,1e-15]),1-1e-15]) for x in predicted]  # within (0,1) interval
    return -(1.0/len(target))*sum([target[i]*log(predicted[i]) +(1.0-target[i])*log(1.0-predicted[i]) for i in range(len(target))])

In [14]:
def Final_fun_1(log_files,member_file,trans_files,msno_file,last_date,last_month,days,date):
    name = 'query'
    msno_df = pd.read_csv(msno_file)
    member_df =  pd.read_csv(member_file)
    Train_ = preprocess_member(member_df,msno_df)
    np.save("{}_member".format(name),Train_)
    preprocess_log(log_files,msno_file,name,last_date,last_month,days)
    Transaction_preprocess(trans_files,msno_file,last_date,name,date)
    prepare_data('query',date)
    query_data = np.load("X_query.npy")
    lgb_model= lgb.Booster(model_file='best model')
    lgb_pred = lgb_model.predict(query_data)
    query = pd.read_csv(msno_file)[['msno','is_churn']]
    query['is_churn'] = lgb_pred.clip(0.+1e-15, 1-1e-15) 
    query[['msno','is_churn']].to_csv('lgb_result.csv', index = False)
    return query[['msno','is_churn']]

In [15]:
def Final_fun_2():
    print("Enter the Log files name:")
    log_files = list(i.strip() for i in input().split())
    print("Enter the msno files name:")
    msno_file = input()
    print("Enter the member files name:")
    member_file = input()
    print("Enter the last date of transaction:")
    last_date = int(input())
    print("Enter the last month of transaction:")
    last_month = tuple(int(i.strip()) for i in input().split(","))
    print("Enter the last 15 days of transaction:")
    days = tuple(int(i.strip()) for i in input().split(","))
    print("Enter the transaction files name:")
    trans_files = list(i.strip() for i in input().split())
    print("Enter the membership expirationdate:")
    date = tuple(int(i.strip()) for i in input().split(","))
    y_ = Final_fun_1(log_files,member_file,trans_files,msno_file,last_date,last_month,days,date)
    y_pred = y_['is_churn']
    yes = input("Do you want to calculate log loss?[y/n]:")
    if 'y' in yes.lower():
        y = pd.read_csv(msno_file).is_churn.astype(int)            
        return log_loss(y_pred,y)

In [18]:
Final_fun_2()

Enter the Log files name:
query_log.csv
Enter the msno files name:
query_msno.csv
Enter the member files name:
query_member.csv
Enter the last date of transaction:
20170131
Enter the last month of transaction:
20170101, 20170131
Enter the last 15 days of transaction:
20170116, 20170131
Enter the transaction files name:
query_trans.csv
Enter the membership expirationdate:
20170201,20170228


100%|██████████| 1/1 [00:00<00:00, 1448.31it/s]

1. total number of cancel feature
2. Last Before transaction features
3. Final transaction feature
 Last 5 Churn and total churn and not churn rate
4. Total_churn_and_not_churn_count:
5. Last_5_not_churns:
additinal feature genration
6. Discount feature 
7. pay per day
8. Sum_count_and_average_features:



100%|██████████| 1/1 [00:00<00:00, 2184.53it/s]


9. slast five features
10. Final Transaction Additional Features
11. number of days from transaction date to membership expire date
12. Days_since_final_transact:
13. Days left
14. Payment_plan_days_subract_interval:
user log (1, 88)
members (1, 33)
trans (1, 135)
X_query (1, 256)
Do you want to calculate log loss?[y/n]:y


0.009527622681534325

In [17]:
#input required for above final method
#     log_files = ["query_log.csv"]
#     msno_file = "query_msno.csv"
#     member_file = "query_member.csv"
#     last_date = 20170131
#     last_month = (20170101, 20170131)
#     days = (20170116, 20170131)
#     trans_files = ["query_trans.csv"]
#     date = (20170201,20170228)